In [ ]:
# Q2. Customer Dataset 

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, length, desc
from pyspark.sql.types import StructType, StructField, StringType

# Initializing  Spark session with Hive support for using SQL in a better way 
spark = SparkSession.builder \
    .appName("CustomerDataAnalysis") \
    .enableHiveSupport() \
    .getOrCreate()

# Since the headers are not present we will manually add the headers in the code which is provided in the question 
# (cust_id,cust_fname,cust_lname,cust_email,cust_password,cust_street,cust_city,cust_state,cust_zipcode)
# Define the correct schema manually.
#  
# Manually defining the column names for the csv 
schema = StructType([
    StructField("cust_id", StringType(), True),
    StructField("cust_fname", StringType(), True),
    StructField("cust_lname", StringType(), True),
    StructField("cust_email", StringType(), True),
    StructField("cust_password", StringType(), True),
    StructField("cust_street", StringType(), True),
    StructField("cust_city", StringType(), True),
    StructField("cust_state", StringType(), True),
    StructField("cust_zipcode", StringType(), True)
])

file_path = r"C:\Users\sayed\Desktop\DDP\M2prac\data\part-00000 (1)"
df = spark.read.option("header", "false").schema(schema).csv(file_path)

# Display the corrected schema
df.printSchema()

# Check if the first row is incorrect (i.e., a misplaced header)
df.show(5)  # Inspect the first few rows

# If the first row is incorrect (not actual data), remove it
first_row = df.limit(1)
df = df.subtract(first_row)

# Create a temporary view for Spark SQL
df.createOrReplaceTempView("customers")

# 1. Find the top 5 most common last names
top_last_names = spark.sql("""
    SELECT cust_lname, COUNT(cust_id) AS count
    FROM customers
    GROUP BY cust_lname
    ORDER BY count DESC
    LIMIT 5
""")
top_last_names.show()

# 2. Find invalid zip codes (not 5 digits)
invalid_zipcodes = spark.sql("""
    SELECT * 
    FROM customers
    WHERE LENGTH(cust_zipcode) != 5 OR cust_zipcode RLIKE '[^0-9]'
""")
invalid_zipcodes.show()

# 3. Count customers per city in California (CA)
cust_count_CA = spark.sql("""
    SELECT cust_city, COUNT(cust_id) AS customer_count
    FROM customers
    WHERE cust_state = 'CA'
    GROUP BY cust_city
    ORDER BY customer_count DESC
""")
cust_count_CA.show()

# Save results to Hive for further use
top_last_names.write.mode("overwrite").saveAsTable("top_last_names")
invalid_zipcodes.write.mode("overwrite").saveAsTable("invalid_zipcodes")
cust_count_CA.write.mode("overwrite").saveAsTable("cust_count_CA")

# Stopping  Spark session
spark.stop()


root
 |-- cust_id: string (nullable = true)
 |-- cust_fname: string (nullable = true)
 |-- cust_lname: string (nullable = true)
 |-- cust_email: string (nullable = true)
 |-- cust_password: string (nullable = true)
 |-- cust_street: string (nullable = true)
 |-- cust_city: string (nullable = true)
 |-- cust_state: string (nullable = true)
 |-- cust_zipcode: string (nullable = true)

+-------+----------+----------+----------+-------------+--------------------+-----------+----------+------------+
|cust_id|cust_fname|cust_lname|cust_email|cust_password|         cust_street|  cust_city|cust_state|cust_zipcode|
+-------+----------+----------+----------+-------------+--------------------+-----------+----------+------------+
|      1|   Richard| Hernandez| XXXXXXXXX|    XXXXXXXXX|  6303 Heather Plaza|Brownsville|        TX|       78521|
|      2|      Mary|   Barrett| XXXXXXXXX|    XXXXXXXXX|9526 Noble Embers...|  Littleton|        CO|       80126|
|      3|       Ann|     Smith| XXXXXXXXX|  